In [27]:
import os

In [28]:
import os
os.chdir(r"C:\Users\p1a2r\OneDrive\Desktop\Git Hub Projects\Text-Summarizer")

In [29]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [30]:
!pip install -qqq python-box
!pip install -qqq PyYAML
!pip install ensure


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
from src.text_summarizer.constants import *
from src.text_summarizer.utils.common import read_yaml, create_directories

In [32]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        
        # Check if config files exist
        if not os.path.exists(config_filepath):
            raise FileNotFoundError(f"Configuration file not found: {config_filepath}")
        if not os.path.exists(params_filepath):
            raise FileNotFoundError(f"Parameters file not found: {params_filepath}")
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # Check if artifacts_root exists in config
        if not hasattr(self.config, 'artifacts_root'):
            raise KeyError("'artifacts_root' key not found in config.yaml. Please add: artifacts_root: artifacts")
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        # Check if data_ingestion section exists
        if not hasattr(self.config, 'data_ingestion'):
            raise KeyError("'data_ingestion' section not found in config.yaml")
        
        config = self.config.data_ingestion
        
        # Check required keys
        required_keys = ['root_dir', 'source_URL', 'local_data_file', 'unzip_dir']
        for key in required_keys:
            if not hasattr(config, key):
                raise KeyError(f"'{key}' not found in data_ingestion section of config.yaml")
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=str(config.source_URL),
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        
        return data_ingestion_config


In [33]:
import urllib.request as request
import zipfile
from src.text_summarizer.logger import logger
from src.text_summarizer.utils.common import get_size

In [34]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            # Create parent directories if they don't exist
            os.makedirs(os.path.dirname(self.config.local_data_file), exist_ok=True)
            
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=str(self.config.local_data_file)
            )
            logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(self.config.local_data_file)}")
            
    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        try:
            with zipfile.ZipFile(str(self.config.local_data_file), "r") as zip_ref:
                zip_ref.extractall(str(unzip_path))
            logger.info(f"Extracted zip file to: {unzip_path}")
        except zipfile.BadZipFile:
            logger.error(f"Bad zip file: {self.config.local_data_file}")
            raise
        except Exception as e:
            logger.error(f"Error extracting zip file: {str(e)}")
            raise

# Main execution
def run_data_ingestion():
    try:
        config = ConfigurationManager()
        data_ingestion_config = config.get_data_ingestion_config()
        data_ingestion = DataIngestion(config=data_ingestion_config)
        data_ingestion.download_file()
        data_ingestion.extract_zip_file()
        logger.info("Data ingestion completed successfully!")
        
    except Exception as e:
        logger.error(f"Data ingestion failed: {str(e)}")
        raise e

if __name__ == "__main__":
    run_data_ingestion()

[2025-09-07 23:52:59,461: INFO: common]: yaml file: config\config.yaml loaded successfully
[2025-09-07 23:52:59,461: INFO: common]: yaml file: config\params.yaml loaded successfully
[2025-09-07 23:52:59,461: INFO: common]: Directory created at: artifacts
[2025-09-07 23:52:59,461: INFO: common]: Directory created at: artifacts/data_ingestion
[2025-09-07 23:53:04,585: INFO: 872128415]: artifacts\data_ingestion\data.zip download with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: EA6B:357B7:F67926:14A1A77:68BE7C69
Accept-Ranges: bytes
Date: Mon, 08 Sep 2025 06:52:59 GMT
Via: 1.1 varnish
X-Served-By: cache-bur-kbur8200119-B